In [1]:
import pandas as pd
import numpy as np

In [8]:
! wget https://research.google.com/youtube8m/csv/train-labels-histogram.csv

--2016-10-22 16:59:15--  https://research.google.com/youtube8m/csv/train-labels-histogram.csv
Resolving research.google.com (research.google.com)... 216.58.217.174, 2607:f8b0:4004:80a::200e
Connecting to research.google.com (research.google.com)|216.58.217.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘train-labels-histogram.csv’

    [ <=>                                   ] 212,516     --.-K/s   in 0.01s   

2016-10-22 16:59:15 (19.3 MB/s) - ‘train-labels-histogram.csv’ saved [212516]



In [2]:
import zipfile
import sys
import csv
from csv import reader

csv.field_size_limit(sys.maxsize)

class DataSampleGenerator():
    def __init__(self,myzipfile, rand_var, frac):
        self.myzip = zipfile.ZipFile(myzipfile, 'r')
        self.rand_var = rand_var
        self.frac = frac
        pass
    
    def transform(self, filename):
        myfile = filename
        with self.myzip.open(myfile) as f:
            df = pd.read_csv(f)
            df = df.sample(frac=self.frac, random_state=self.rand_var)
            return df 

In [26]:
trans = DataSampleGenerator('YT8M-ids/training_ids.zip', 0, 0.0001) # 0.01% of data

In [27]:
video_ids = trans.transform('training_ids.txt')

In [28]:
video_ids.columns = ['yt_id']

In [35]:
video_ids['yt_id'].values

array(['V8mPzXthHBE', 'cLLV93ZazlU', 'OdReEevl84s', ..., 'RR!', 'i-!',
       'TbwSmO0a2zg'], dtype=object)

In [13]:
label_ids = pd.read_csv('train-labels-histogram.csv')

In [56]:


class DataIngestor():
    def __init__(self,video_ids, config_file):
        import argparse
        import ConfigParser
        config = ConfigParser.ConfigParser()
        config.read(config_file)
        yt_config = dict(config.items('Youtube')) 
        self.api_key = yt_config['api_key']
        self.video_ids = video_ids
        pass
    def api_call(self):
        import requests
        url_pages = ['https://www.googleapis.com/youtube/v3/videos?part=statistics,snippet&id=%s&key=%s'%(video_id,self.api_key) for video_id in video_ids['yt_id']]
        self.responses = [requests.get(url) for url in url_pages]
        return self.responses
    
    def build_df(self):
        import pandas as pd
        from pandas.io.json import json_normalize
        frames = [json_normalize(response.json()['items']) for response in self.responses if response.json()['items']]
        self.df =pd.concat([pd.DataFrame(frame)for frame in frames])
        return self.df




In [59]:
trans = DataIngestor(video_ids, '.ttapis.cfg')

In [60]:
trans.api_call()
trans.build_df()

,etag,id,kind,snippet.categoryId,snippet.channelId,snippet.channelTitle,snippet.defaultAudioLanguage,snippet.description,snippet.liveBroadcastContent,snippet.localized.description,...,snippet.thumbnails.medium.width,snippet.thumbnails.standard.height,snippet.thumbnails.standard.url,snippet.thumbnails.standard.width,snippet.title,statistics.commentCount,statistics.dislikeCount,statistics.favoriteCount,statistics.likeCount,statistics.viewCount
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/Vg1tvI_HtR7hRhuWS...",V8mPzXthHBE,youtube#video,10,UCgX0qIG3zdKwWx7sZRRcElw,AmyWinehousePL,NaN,Amy Winehouse @ 90th Birthady Nelson Mandela\r...,none,Amy Winehouse @ 90th Birthady Nelson Mandela\r...,...,320,NaN,NaN,NaN,Amy Winehouse - Rehab (90th Birthday Nelson Ma...,5,2,0,31,10654
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/0c1Q5itz3IngdWaLW...",cLLV93ZazlU,youtube#video,26,UCQlGBspQdj17WOPBQMT1k9A,expertvillage,NaN,How to prepare dry yeast for inoculation when ...,none,How to prepare dry yeast for inoculation when ...,...,320,NaN,NaN,NaN,Home Brewing India Pale Ale (IPA) Beer : Prepa...,0,2,0,2,2444
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/fR92IGB31bRzUgoJh...",OdReEevl84s,youtube#video,2,UCMfrFOBruMKq7wqgJzkXvEA,JTHughesCars,NaN,CV60LSX Honda Civic Type S i-VTEC 1.8 metallic...,none,CV60LSX Honda Civic Type S i-VTEC 1.8 metallic...,...,320,480.0,https://i.ytimg.com/vi/OdReEevl84s/sddefault.jpg,640.0,Honda Civic Type S CV60LSX,1,1,0,8,6934
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/wjqoKWcdCaXPPdtzS...",GlFd9KkY8yk,youtube#video,26,UCeI_s6u0CJY89oV8IzW9aIQ,Sintoru09,NaN,AWESOME,none,AWESOME,...,320,NaN,NaN,NaN,Best Stamina Type Beyblade (Must See),11,14,0,9,2240
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/7To-PTCWvXwYeuPcT...",7AkD4HMYcxo,youtube#video,10,UCKfzTbDptPkulLy0sl8b40g,Dive Band,NaN,"This is the official lyric video for ""Last Cal...",none,"This is the official lyric video for ""Last Cal...",...,320,480.0,https://i.ytimg.com/vi/7AkD4HMYcxo/sddefault.jpg,640.0,DIVE - Last Call Romance (Official Lyric Video),14,3,0,135,6710
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/XePuJiBrNIFfa7C1D...",WckqiWoRcRg,youtube#video,10,UCmMjFGb7OOJrcxlyNjL5dvQ,Trumpetralfino Willing,NaN,Für alle Katzen Freunde meine Aufnahme zu spaß...,none,Für alle Katzen Freunde meine Aufnahme zu spaß...,...,320,NaN,NaN,NaN,Katzen Song Alley Cat Orchester Ralf Willing,4,1,0,22,2881
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/T2Lv6cCXaWO__8oFo...",pOQPU41pTlI,youtube#video,22,UC3bu-wgMga6_XgkK5FvTdfQ,Frogmaster Riviera,en,First scuba detection dive with stock Minelab ...,none,First scuba detection dive with stock Minelab ...,...,320,480.0,https://i.ytimg.com/vi/pOQPU41pTlI/sddefault.jpg,640.0,Frogmaster #2 Scuba detection diving Minelab E...,32,8,0,44,61697
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/Pb4jcJ41WekOB2enl...",kOLYqt3lgRM,youtube#video,20,UCkpnaL7XKTt1SLYiU1m2xoQ,Arcane Penguin Gaming,en-GB,"Alex, Fred and Junior show you how to unlock t...",none,"Alex, Fred and Junior show you how to unlock t...",...,320,480.0,https://i.ytimg.com/vi/kOLYqt3lgRM/sddefault.jpg,640.0,GTA San Andreas Achievements - Hustle Some & W...,1,1,0,18,1915
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/2MoRPmdOk58jTxs9E...",O2tdhMXX4lU,youtube#video,28,UCrHTJLe6ujE13YSe-Wt6Ltg,Lord Vader,NaN,A Friend send me the Body Glove for my Htc HD2...,none,A Friend send me the Body Glove for my Htc HD2...,...,320,480.0,https://i.ytimg.com/vi/O2tdhMXX4lU/sddefault.jpg,640.0,Htc HD2 Holster,4,1,0,1,2583
0,"""I_8xdZu766_FSaexEaDXTIfEWc0/bDFQWW4XScJ3Ygyhn...",7pHVSPLxqiQ,youtube#video,17,UC-RE7J_WNFCDzy7ZoxTT_CA,Amelia Brodka,NaN,skateboarding 2005/2006,none,skateboarding 2005/2006,...,320,NaN,NaN,NaN,I like to skateboard (2005/2006),2,1,0,7,1641


In [55]:
trans.df.columns

Index([                              u'etag',
                                       u'id',
                                     u'kind',
                       u'snippet.categoryId',
                        u'snippet.channelId',
                     u'snippet.channelTitle',
             u'snippet.defaultAudioLanguage',
                      u'snippet.description',
             u'snippet.liveBroadcastContent',
            u'snippet.localized.description',
                  u'snippet.localized.title',
                      u'snippet.publishedAt',
                             u'snippet.tags',
        u'snippet.thumbnails.default.height',
           u'snippet.thumbnails.default.url',
         u'snippet.thumbnails.default.width',
           u'snippet.thumbnails.high.height',
              u'snippet.thumbnails.high.url',
            u'snippet.thumbnails.high.width',
         u'snippet.thumbnails.maxres.height',
            u'snippet.thumbnails.maxres.url',
          u'snippet.thumbnails.max